In [ ]:
# ==========================================
# 0️⃣ นำเข้าคลังข้อมูลที่จำเป็น
# ==========================================
import pandas as pd
import numpy as np
import time
from cassandra.cluster import Cluster
from datetime import datetime
import plotly.express as px
from settrade_v2 import Investor

In [ ]:
# ==========================================
# 1️⃣ เชื่อมต่อ Cassandra และสร้าง keyspace + table
# ==========================================
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

session.execute("""
    CREATE KEYSPACE IF NOT EXISTS data_stock
    WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1};
""")
session.set_keyspace('data_stock')

session.execute("""
    CREATE TABLE IF NOT EXISTS candlestick_data (
        symbol text,
        time timestamp,
        open_price float,
        high_price float,
        low_price float,
        close_price float,
        volume bigint,
        value float,
        PRIMARY KEY (symbol, time)
    ) WITH CLUSTERING ORDER BY (time ASC);
""")
print("✅ Keyspace และ Table พร้อมใช้งาน!")

✅ Keyspace และ Table พร้อมใช้งาน!


In [ ]:
# ==========================================
# 2️⃣ เชื่อมต่อ Settrade API
# ==========================================
investor = Investor(
    app_id="kc2VLfCa7PkZSe0W",
    app_secret="aL0MiKbupfsjWDzoRz1lsSgUYy68HkJYjqP2aLMROnI=",
    broker_id="SANDBOX",
    app_code="SANDBOX",
    is_auto_queue=False
)
market = investor.MarketData()

In [ ]:
# ==========================================
# 3️⃣ โหลดรายชื่อหุ้น
# ==========================================
try:
    symbols_df = pd.read_excel("get_stock_name/stocth_names.xlsx")
    symbols = symbols_df['หลักทรัพย์'].dropna().tolist()
except FileNotFoundError:
    print("❌ ไม่พบไฟล์ stocth_names.xlsx ใช้รายชื่อหุ้นที่กำหนดแทน")
    symbols = ["PTT", "AOT", "SCB", "CPALL", "ADVANC"]

In [ ]:
# ==========================================
# 4️⃣ ฟังก์ชันดึงข้อมูล Candlestick
# ==========================================
def get_candlestick_data(symbol):
    try:
        res = market.get_candlestick(
            symbol=symbol,
            interval="1d",
            limit=100,
            normalized=True
        )
        if isinstance(res, dict) and "data" in res:
            return res["data"]
        else:
            return res
    except Exception as e:
        print(f"⚠️ ไม่สามารถดึงข้อมูลของ {symbol}: {e}")
        return None


In [ ]:
# ==========================================
# 5️⃣ ฟังก์ชันบันทึกลง Cassandra
# ==========================================
def insert_financial_data(symbol, res):
    if not res or "time" not in res:
        print(f"⚠️ ไม่มีข้อมูลสำหรับ {symbol}")
        return

    for i in range(len(res["time"])):
        timestamp = datetime.fromtimestamp(res["time"][i])
        session.execute("""
            INSERT INTO candlestick_data (
                symbol, time, open_price, high_price, low_price,
                close_price, volume, value
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            symbol,
            timestamp,
            res["open"][i],
            res["high"][i],
            res["low"][i],
            res["close"][i],
            res["volume"][i],
            res["value"][i]
        ))
    print(f"✅ เพิ่มข้อมูล {len(res['time'])} รายการของหุ้น {symbol} สำเร็จ!")

In [ ]:
# ==========================================
# 6️⃣ ดึงและบันทึกข้อมูลหุ้นทั้งหมด
# ==========================================
for symbol in symbols:
    print(f"\n📦 กำลังโหลดข้อมูล {symbol} ...")
    res = get_candlestick_data(symbol)
    insert_financial_data(symbol, res)
    time.sleep(2)

print("\n🎉 เสร็จสิ้นการโหลดข้อมูลทั้งหมด!")


📦 กำลังโหลดข้อมูล 24CS ...
✅ เพิ่มข้อมูล 100 รายการของหุ้น 24CS สำเร็จ!

📦 กำลังโหลดข้อมูล 2S ...
✅ เพิ่มข้อมูล 100 รายการของหุ้น 2S สำเร็จ!

📦 กำลังโหลดข้อมูล 3BBIF ...
✅ เพิ่มข้อมูล 100 รายการของหุ้น 3BBIF สำเร็จ!

📦 กำลังโหลดข้อมูล A ...
✅ เพิ่มข้อมูล 100 รายการของหุ้น A สำเร็จ!

📦 กำลังโหลดข้อมูล A5 ...
✅ เพิ่มข้อมูล 100 รายการของหุ้น A5 สำเร็จ!

📦 กำลังโหลดข้อมูล AAI ...
✅ เพิ่มข้อมูล 100 รายการของหุ้น AAI สำเร็จ!

📦 กำลังโหลดข้อมูล AAV ...
✅ เพิ่มข้อมูล 100 รายการของหุ้น AAV สำเร็จ!

📦 กำลังโหลดข้อมูล ABM ...
✅ เพิ่มข้อมูล 100 รายการของหุ้น ABM สำเร็จ!

📦 กำลังโหลดข้อมูล ACAP ...
✅ เพิ่มข้อมูล 100 รายการของหุ้น ACAP สำเร็จ!

📦 กำลังโหลดข้อมูล ACC ...
✅ เพิ่มข้อมูล 100 รายการของหุ้น ACC สำเร็จ!

📦 กำลังโหลดข้อมูล ACE ...
✅ เพิ่มข้อมูล 100 รายการของหุ้น ACE สำเร็จ!

📦 กำลังโหลดข้อมูล ACG ...
✅ เพิ่มข้อมูล 100 รายการของหุ้น ACG สำเร็จ!

📦 กำลังโหลดข้อมูล ADB ...
✅ เพิ่มข้อมูล 100 รายการของหุ้น ADB สำเร็จ!

📦 กำลังโหลดข้อมูล ADD ...
✅ เพิ่มข้อมูล 100 รายการของหุ้น ADD สำเร็จ!

📦 กำล

In [ ]:
# ==========================================
# 7️⃣ วิเคราะห์คุณภาพหุ้น (สมมติข้อมูลพื้นฐานจำลอง)
# ==========================================
data_quality = pd.DataFrame({
    "Symbol": symbols,
    "P/E": np.random.uniform(5, 40, len(symbols)),
    "P/BV": np.random.uniform(0.5, 5, len(symbols)),
    "ROE": np.random.uniform(2, 30, len(symbols)),
    "Dividend": np.random.uniform(0, 8, len(symbols))
})

def classify_quality(row):
    if row["ROE"] > 20 and row["P/E"] < 15 and row["P/BV"] < 2:
        return "Excellent"
    elif row["ROE"] > 15 and row["P/E"] < 20:
        return "Good"
    elif row["ROE"] > 10:
        return "Fair"
    elif row["ROE"] > 5:
        return "Poor"
    else:
        return "Unacceptable"

data_quality["Quality"] = data_quality.apply(classify_quality, axis=1)

In [18]:
# ==========================================
# 8️⃣ Treemap Divining Based on Quality + Legend อธิบายสี
# ==========================================
treemap_data = data_quality.copy()
treemap_data["Size"] = treemap_data["ROE"]  # ขนาดกล่อง = ROE

# Mapping สีตามระดับ Quality
color_map = {
    "Excellent": "green",
    "Good": "limegreen",
    "Fair": "gold",
    "Poor": "orange",
    "Unacceptable": "red"
}

fig = px.treemap(
    treemap_data,
    path=["Symbol"],      # ถ้ามี Sector: ["Sector","Symbol"]
    values="Size",
    color="Quality",      # ใช้ชื่อ Quality ตรง ๆ
    color_discrete_map=color_map,
    title="📊 Divining Based on Quality: Stock Heatmap",
    custom_data=["ROE", "P/E", "P/BV", "Dividend"]
)

# Tooltip แสดง ROE, P/E, P/BV, Dividend
fig.update_traces(
    texttemplate="%{label}<br>ROE: %{customdata[0]:.2f}",
    hovertemplate=(
        "<b>%{label}</b><br>" +
        "ROE: %{customdata[0]:.2f}<br>" +
        "P/E: %{customdata[1]:.2f}<br>" +
        "P/BV: %{customdata[2]:.2f}<br>" +
        "Dividend: %{customdata[3]:.2f}<extra></extra>"
    )
)

# Legend ชัดเจนด้านข้าง
fig.update_layout(
    legend_title=dict(
        text='📊 Stock Quality Level',
        font=dict(size=15, color="black")
    ),
    legend=dict(
        orientation="v",
        yanchor="top",
        y=1,
        xanchor="right",
        x=1.02,
        title_font_size=12,
        font=dict(size=12),
        itemsizing='constant',
        traceorder='normal'
    )
)

# เพิ่ม annotation อธิบายสีด้านข้าง (ถ้าต้องการ)
annotations = [
    dict(x=1.1, y=1, xref='paper', yref='paper',
         text="<b>Color Legend:</b><br>Green = Excellent<br>Lime = Good<br>Gold = Fair<br>Orange = Poor<br>Red = Unacceptable",
         showarrow=False, align='left', font=dict(size=6))
]
fig.update_layout(annotations=annotations)

fig.show()


In [19]:
# ==========================================
# 9️⃣ สรุปข้อมูลหุ้นตามคุณภาพ
# ==========================================
summary = data_quality.groupby("Quality").agg({
    "Symbol": "count",
    "ROE": ["mean", "max", "min"],
    "P/E": ["mean", "max", "min"],
    "P/BV": ["mean", "max", "min"],
    "Dividend": ["mean", "max", "min"]
}).reset_index()

summary.columns = ["Quality", "Count", "ROE_mean", "ROE_max", "ROE_min",
                   "PE_mean", "PE_max", "PE_min",
                   "PBV_mean", "PBV_max", "PBV_min",
                   "Dividend_mean", "Dividend_max", "Dividend_min"]

cols_to_show = [
    "Quality", "Count",
    "ROE_mean", "ROE_max", "ROE_min",
    "PE_mean", "PE_max", "PE_min",
    "PBV_mean", "PBV_max", "PBV_min",
    "Dividend_mean", "Dividend_max", "Dividend_min"
]

print("📋 สรุปข้อมูลหุ้นตามคุณภาพ:")
print(summary[cols_to_show])

📋 สรุปข้อมูลหุ้นตามคุณภาพ:
        Quality  Count   ROE_mean    ROE_max    ROE_min    PE_mean     PE_max  \
0     Excellent     40  24.239191  29.961172  20.294811   9.796820  14.887022   
1          Fair    422  18.924023  29.930384  10.012438  27.678325  39.898774   
2          Good    194  22.118190  29.995659  15.118021  12.748192  19.791613   
3          Poor    167   7.589316   9.982014   5.021706  22.095639  39.941136   
4  Unacceptable    100   3.309604   4.988172   2.000268  21.903768  39.700060   

     PE_min  PBV_mean   PBV_max   PBV_min  Dividend_mean  Dividend_max  \
0  5.008258  1.213453  1.962482  0.517377       3.727488      7.787320   
1  5.891495  2.849913  4.991726  0.518899       4.127206      7.975746   
2  5.028185  3.112726  4.983473  0.510380       4.033786      7.916271   
3  5.004009  2.964877  4.998078  0.503963       4.025593      7.974645   
4  5.007666  2.668443  4.971239  0.529188       4.111796      7.988479   

   Dividend_min  
0      0.382583  
1    